In [1]:
import numpy as np
from sympy import Matrix
#from numpy.linalg import matrix_rank

height = 4
width = 4

def normal_toggling(y_toggle, x_toggle):
    toggle = np.zeros((height, width))
    
    toggle[y_toggle][x_toggle] = 1
    
    if (y_toggle - 1 >= 0):
        toggle[y_toggle - 1][x_toggle] = 1
    if (y_toggle + 1 <= height - 1):
        toggle[y_toggle + 1][x_toggle] = 1
    if (x_toggle - 1 >= 0):
        toggle[y_toggle][x_toggle - 1] = 1
    if (x_toggle + 1 <= width - 1):
        toggle[y_toggle][x_toggle + 1] = 1
    
    return toggle

def normal_building(height, width):
    adjacency = np.zeros((0, height*width))
    
    for i in range(height):
        for j in range(width):
            toggle = normal_toggling(i, j)
            row = toggle.flatten()
            adjacency = np.vstack((adjacency,row))
    
    return adjacency
    

adjacency = Matrix(normal_building(height, width))
#print(adjacency)
rank = adjacency.rank() 
print(rank)

#NoC = np.size(adjacency,1)
#print(NoC)
#rank = matrix_rank(adjacency)
#print(rank)
#nullity = NoC - rank

#print(adjacency)
#print(rank)

14


In [12]:
# @mfunction("ahat")
def Ahat(n=None):
    # Creates the adjacency linegraph/hat matrix of size n
    A = eye(n)# Start by putting one's on diagonal
    for x in mslice[1:n]:    # Place 1's in upper diagonal
        if x > 1:
            A(x, x - 1).lvalue = 1
        end
        if x < n:        # Place 1's in lower diagonal
            A(x, x + 1).lvalue = 1
    ahat = A

In [14]:
# @mfunction("adjmatrix")
def GenAdj(n=None):
    # Generates an adjacency matrix for square game boards with nxn vertices with + shape
    edges()
    # Create identity matrix size n
    I = eye(n)
    # Create A (linegraph/hat matrix)
    A = eye(n)# Start by putting one's on diagonal
    for x in mslice[1:n]:    # Place 1's in upper diagonal
        if x > 1:
            A(x, x - 1).lvalue = 1
        end
        if x < n:        # Place 1's in lower diagonal
            A(x, x + 1).lvalue = 1
    # Build M, the adjacency matrix, using I and A
    m = n * n# Dimension of A
    M = zeros(m)# Start out with all zeros as placeholders
    for x in mslice[1:n]:    # essentially inputting matricies at each n entry
        for y in mslice[1:n]:
            if x == y:            #along the diagonal, copy the A matrix entries
                M = copy1(M, A, x, y, n)
            elif abs(x - y) == 1:            # along the upper and lower diag copy I
                M = copy1(M, I, x, y, n)
    adjmatrix = M

In [18]:
# @mfunction("q")
def copy1(src=None, hat=None, locx=None, locy=None, n=None):
    #
    #src is M that was set up initally as all 0's (changing at each step)
    #hat is either A or I
    #locx and locy correspond to the 1-n index
    #changes nxn matrix to n^2xn^2 matrix
    #copies hat(n,n) into src(n^2,n^2)
    i = 1#n based row
    for a in mslice[(locx - 1) * n + 1:locx * n]:    #n*n based row
        j = 1    #n based column
        for b in mslice[(locy - 1) * n + 1:locy * n]:        #n*n based column
            src(a, b).lvalue = hat(i, j)
            j = j + 1
        i = i + 1
    q = src

In [21]:
# @mfunction("invsol")
def InvSol(n=None):
    # Finds the inverse or pseduo inverse of the adjacency matrix with the following specifications:
    # nxn dimension game boards with cross pattern edges with light state set size 2 (on or off)
    # Also outputs the number of quiet patterns in the game board
    # Need GenAdj function code to run this function
    M = GenAdj(n)# GenAdj is the function written to generate the adjacency matrix for an nxn game board
    m = n * n# The size of the adjacency matrix is actually n^2 (this is the number of vertices)
    M = mod(M, 2)# Ensure inputs are mod 2, as they should already be
    M = mcat([M, eye(m)])# Augment adjacency matrix with identity matrix
    #FORWARD ELIMINATION (mod 2 simplified):
    # Moving through columns from left to right, ensures a 1 in the pivot position when 
    # possible and eliminates all 1's below the pivot using row operations
    startback = 0# Used to determine when to start backward elimination
    finalcol = 0# Used to determine which was the last non-zero column (rank of matrix)
    for col in mslice[1:m]:    # Scan through columns left to right
        for row in mslice[col:m]:        # Scan through rows from diagonal entry down
            if row == col:            # In diagonal position: create pivots
                if M(row, col) == 1:                # If a 1 already in the diagonal, pivot already there
                    finalcol = col                # Update status of rank of the matrix
                    continue

                else:                # If a one is not in the diagonal
                    foundit = 0                # Used to advance on to backward elimination if no swap is possible
                    for row2 in mslice[row + 1:m]:                    # Scan down the rows below the diagonal
                        if M(row2, col) == 1:                        # If a one is found below the diagonal
                            temp = M(row, mslice[:])                        # Swap this row and the pivot row
                            M(row, mslice[:]).lvalue = M(row2, mslice[:])
                            M(row2, mslice[:]).lvalue = temp
                            foundit = 1                        # Swapped, now move on to eliminate ones below the pivot and continue forward elimination
                            break

                    if foundit == 0:                    # If there is not a 1 below the diagnoal, no pivot possible
                        # Start backward elimination
                        startback = 1

                        break                                            # Break out of forward elimination

            else:            # Eliminating ones below the pivot
                if M(row, col) == 0:                # If already a zero entry, no change necessary
                    continue
                else:                # For rows with one entries, replace this row with the sum of current row and pivot row
                    M(row, mslice[:]).lvalue = mod(M(col, mslice[:]) + M(row, mslice[:]), 2)

        if startback == 1:
            break                    # Break out of forward eliminaton
    #Displays the rank and number of quiet patterns
    str = fprintf(mstring('Rank = %d\\n'), finalcol)
    str = fprintf(mstring('Nullity = %d\\n'), m - finalcol)
    #BACKWARD ELIMINATION (mod 2 simiplified):
    # Moves along columns right to left eliminating ones above each pivot, creating a
    # identity matrix on the left (or as close as possible)
    for col in mslice[finalcol:-1:1]:    # Scan through columns right to left
        if M(col, col) != 1:        # Assert that the pivot is a 1 (should be at this point)
            disp(mstring('Non-zero pivot at this column in back elimination'))
            col()        # Display column where this error occurs
            invsol = M
            return
        for row in mslice[col - 1:-1:1]:        # Scan through rows from one above diagonal, to the top of the matrix
            if M(row, col) != 0:            # When there is a 1, eliminate the 1 by replacing row with sum of current row and pivot row
                M(row, mslice[:]).lvalue = mod(M(col, mslice[:]) + M(row, mslice[:]), 2)
    invsol = M

In [22]:
# @mfunction("prref")
def Prref(a=None, n=None):
    #Row reduces any square matrix a (must input the dimension n) using mod 2
    #at each step
    m = n# The size of the adjacency matrix is actually n^2 (this is the number of vertices)
    #FORWARD ELIMINATION (mod 2 simplified):
    # Moving through columns from left to right, ensures a 1 in the pivot position when
    # possible and eliminates all 1's below the pivot using row operations
    startback = 0# Used to determine when to start backward elimination
    finalcol = 0# Used to determine which was the last non-zero column (rank of matrix)
    for col in mslice[1:m]:    # Scan through columns left to right
        for row in mslice[col:m]:        # Scan through rows from diagonal entry down
            if row == col:            # In diagonal position: create pivots
                if a(row, col) == 1:                # If a 1 already in the diagonal, pivot already there
                    finalcol = col                # Update status of rank of the matrix

                    continue

                else:                # If a one is not in the diagonal
                    foundit = 0                # Used to advance on to backward elimination if no swap is possible
                    for row2 in mslice[row + 1:m]:                    # Scan down the rows below the
                        diagonal()
                        if a(row2, col) == 1:                        # If a one is found below the diagonal
                            temp = a(row, mslice[:])                        # Swap this row and the pivot row

                            a(row, mslice[:]).lvalue = a(row2, mslice[:])
                            a(row2, mslice[:]).lvalue = temp
                            foundit = 1                        # Swapped, now move on to eliminate ones below the pivot and continue forward elimination
                            break
                    if foundit == 0:                    # If there is not a 1 below the diagnoal, no pivot possible
                        # Start backward elimination
                        startback = 1

                        break                                            # Break out of forward elimination

            else:            # Eliminating ones below the pivot
                if a(row, col) == 0:                # If already a zero entry, no change necessary
                    continue
                else:                # For rows with one entries, replace this row with the sum of current row and pivot row
                    a(row, mslice[:]).lvalue = mod(a(col, mslice[:]) + a(row, mslice[:]), 2)
        if startback == 1:
            break                    # Break out of forward eliminaton
    #Displays the rank and number of quiet patterns

    #str=fprintf('Rank = %d\n', finalcol);
    #str=fprintf('Nullity = %d\n', m-finalcol);
    #BACKWARD ELIMINATION (mod 2 simiplified):
    # Moves along columns right to left eliminating ones above each pivot, creating a
    # identity matrix on the left (or as close as possible)
    for col in mslice[finalcol:-1:1]:    # Scan through columns right to left
        if a(col, col) != 1:        # Assert that the pivot is a 1 (should be at this point)
            disp(mstring('Non-zero pivot at this column in back elimination'))
            col()        # Display column where this error occurs
            prref = a
            return
        for row in mslice[col - 1:-1:1]:        # Scan through rows from one above diagonal, to the top of the matrix
            if a(row, col) != 0:            # When there is a 1, eliminate the 1 by replacing row with sum of current row and pivot row
                a(row, mslice[:]).lvalue = mod(a(col, mslice[:]) + a(row, mslice[:]), 2)
    prref = a